In [1]:
import os
import random
from osgeo import gdal
import geopandas as gpd
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from geonet import tiler, mask, raster, dataset, utils
from geonet.visualizations import plotImagePair
from PIL import Image

In [14]:
src_image = "../../data/_mvp/vrn_19_coord.tif"
src_vector = "../../data/_mvp/vrn_agro_train.shp"
tiles_dir = "../../data/_mvp/forest2_train_tiles/vrn/"
labels_dir = "../../data/_mvp/forest_train_labels/vrn/"
mask_dir = "../../data/_mvp/forest2_train_masks/vrn/"

In [15]:
raster_tiler = tiler.RasterTiler(dest_dir=tiles_dir, src_tile_size=(1024, 1024), verbose=True, tile_overlay=256)

Initializing Tiler...
Tiler initialized.
dest_dir: ../../data/_mvp/forest2_train_tiles/vrn/
dest_crs will be inferred from source data.
src_tile_size: (1024, 1024)
tile size units metric: False
Resampling is set to None


In [16]:
raster_bounds_crs = raster_tiler.tile(src_image)

Beginning tiling...


Checking input data...
[1, 2, 3, 4, 5, 6]
Source CRS: EPSG:3857
Destination CRS: EPSG:3857
Inputs OK.
count of tile bounds 256
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
reading data from window
None
rea

KeyboardInterrupt: 

In [18]:
vector_tiler = tiler.VectorTiler(dest_dir=labels_dir, verbose=True)
vector_tiler.tile(src_vector,
                  tile_bounds=raster_tiler.tile_bounds,
                  tile_bounds_crs=raster_bounds_crs)

Preparing the tiler...
Initialization done.


Num tiles: 240



In [19]:
def add_prefix(path, prefix):
    files = os.listdir(path)
    for index, file in enumerate(files):
        os.rename(os.path.join(path, file), os.path.join(path, prefix+file))

In [20]:
add_prefix("../../data/_mvp/forest_train_labels/ostrog/", "ostrog_")

In [46]:
images = os.listdir("../../data/_mvp/forest2_train_tiles")
labels = os.listdir("../../data/_mvp/forest_train_labels")
masks = os.listdir("../../data/_mvp/forest2_train_masks")
images.sort()
labels.sort()
masks.sort()
print(len(images))
print(len(labels))
print(len(masks))

662
707
662


In [37]:
images[0:5]

['kursk_3833411_6695462.tif',
 'kursk_3833411_6707884.tif',
 'kursk_3833411_6720305.tif',
 'kursk_3833411_6732727.tif',
 'kursk_3833411_6745148.tif']

In [36]:
labels[0:5]

['.ipynb_checkpoints',
 'kur_geoms_3833411_6695462.geojson',
 'kur_geoms_3833411_6707884.geojson',
 'kur_geoms_3833411_6720305.geojson',
 'kur_geoms_3833411_6732727.geojson']

In [42]:
masks[0:5]

['.ipynb_checkpoints',
 'kursk_3833411_6695462.tif',
 'kursk_3833411_6707884.tif',
 'kursk_3833411_6720305.tif',
 'kursk_3833411_6732727.tif']

In [59]:
import shutil
shutil.rmtree("../../data/_mvp/forest2_test_tiles/.ipynb_checkpoints")

In [40]:
for count, filename in enumerate(os.listdir("../../data/_mvp/forest2_train_masks/t/")): 
    dst = filename.split("_")
    del dst[1]
    dst = '_'.join(dst)
    src ="../../data/_mvp/forest2_train_masks/t/" + filename 
    dst ="../../data/_mvp/forest2_train_masks/t/" + dst 
          
    # rename() function will 
    # rename all the files 
    os.rename(src, dst)

In [29]:
for i in range(0, len(images)):
    gdf = gpd.read_file(labels_dir + '/' + labels[i])
    ref_im = tiles_dir + '/' + images[i]
    fp_mask = mask.footprint_mask(gdf, reference_im=ref_im, shape=(1024, 1024), do_transform="reference_im")
    #b_mask = mask.boundary_mask(fp_mask, boundary_width=5)
    try:
        #c_mask = mask.contact_mask(gdf, reference_im=ref_im, contact_spacing=5)
        #mask_full = np.dstack((fp_mask, b_mask, c_mask))
        mask_im = "../../data/_mvp/forest_train_masks/ostrog/" + images[i]
        Image.fromarray(fp_mask).save(mask_im)
    except:
        os.remove("../../data/_mvp/forest_train_labels/ostrog/" + labels[i])
        os.remove("../../data/_mvp/forest_train_tiles/ostrog/" + images[i])
        print(labels[i] + " processing failed. Files removed")

In [33]:
# split files on train and test
def test_split(train_dir, train_masks_dir, test_dir, test_masks_dir, shape, per=0.3):
    k = np.int(np.round(shape * per))
    test_idx = random.sample(range(0, shape), k)
    file_names = os.listdir(train_dir)
    i = 0
    for file_name in file_names:
        if i in test_idx:
            shutil.move(os.path.join(train_dir, file_name), test_dir)
            shutil.move(os.path.join(train_masks_dir, file_name), test_masks_dir)
        i+=1

In [45]:
test_split("../../data/_mvp/forest2_train_tiles/", "../../data/_mvp/forest2_train_masks/",
          "../../data/_mvp/forest2_test_tiles/", "../../data/_mvp/forest2_test_masks/", shape=946)

In [2]:
# add augmentations
aug = A.Compose([
    #A.Normalize(mean=(0.0095, 0.0087, 0.0078), std=(0.0075, 0.0070, 0.0060)),
    A.RandomRotate90(p=0.6),
    A.HorizontalFlip(p=0.6),
    #ToTensorV2()
])

In [3]:
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def preprocess_input(x, mean=[0.187, 0.182, 0.139, 0.215, 0, 0], std=[0.015, 0.035, 0.038, 0.067, 1, 1], input_space="RGB", input_range=[0,1], **kwargs):

    if input_space == "BGR":
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 5000.0

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.RandomRotate90(p=0.5),
        A.Flip(),
        A.GridDistortion(p=0.4),
        #A.Downscale(scale_min=0.2, scale_max=0.3, p=0.3),
        A.Transpose(),
        A.Lambda(image=preprocess_input),
        A.Lambda(image=to_tensor),
    ]
    return A.Compose(_transform)

In [4]:
def get_inference_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocess_input),
        A.Lambda(image=to_tensor)
    ]
    return A.Compose(_transform)

In [5]:
# initialize model
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['agro']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
    in_channels=6,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [6]:
trainData = dataset.TiledRasterDataset("../../data/_mvp/forest2_train_tiles", "../../data/_mvp/forest2_train_masks", classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))
validData = dataset.TiledRasterDataset("../../data/_mvp/forest2_test_tiles", "../../data/_mvp/forest2_test_masks", classes=["agro"], preprocessing=get_preprocessing(preprocessing_fn))

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [5]:
x_train_file = raster.get_array_from_tiff("../../data/_mvp/kursk_train_3857.tif")
x_train_file = np.dstack(x_train_file)
y_train_file = raster.get_array_from_tiff("../../data/_mvp/kursk_forest_mask.tif")[0]
x_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_train_3857.tif")
x_valid_file = np.dstack(x_valid_file)
y_valid_file = raster.get_array_from_tiff("../../data/_mvp/vrn_3857_forest_mask.tif")[0]
trainData = dataset.RasterDataset(x_train_file.astype(int), y_train_file.astype(int), classes=["agro"], tile_size=1024, step=768, augmentation=aug, preprocessing=get_preprocessing(preprocessing_fn))
validData = dataset.RasterDataset(x_valid_file.astype(int), y_valid_file.astype(int), classes=["agro"], tile_size=1024, step=768, augmentation=aug, preprocessing=get_preprocessing(preprocessing_fn))

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [6]:
# add data loaders
train_loader = DataLoader(trainData, batch_size=1, shuffle=True, num_workers=2)
valid_loader = DataLoader(validData, batch_size=1, shuffle=False, num_workers=2)

NameError: name 'trainData' is not defined

In [8]:
torch.cuda.is_available()

True

In [9]:
loss = smp.utils.losses.BCELoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.AdamW([ 
    dict(params=model.parameters(), lr=0.00015),
])

In [10]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device = DEVICE
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics,
    device = DEVICE
)

In [11]:
model.cuda()

FPN(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu)

In [12]:
# train model
max_score = 0.61

'''checkpoint = torch.load("../geonet/weights/best/forestnet_chekpoint_v1.2 - 0.613902030818754.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']'''

for i in range(0, 100):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        best_model = model
        torch.save(best_model, '../geonet/weights/forestnet-c_bce_v1.3 - {}.pth'.format(max_score))
        torch.save({
            'epoch': i,
            'model_state_dict': best_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '../geonet/weights/forestnet_chekpoint-c_bce_v1.3 - {}.pth'.format(max_score))
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-4
        print('Decrease decoder learning rate to 5e-4!')


Epoch: 0
train:   0%|          | 0/662 [00:00<?, ?it/s]

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1, 1024, 1024])) that is different to the input size (torch.Size([1, 1, 1024, 1024])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


valid: 100%|██████████| 284/284 [00:24<00:00, 11.75it/s, bce_loss - 0.1972, iou_score - 0.5508]

Epoch: 1
valid: 100%|██████████| 284/284 [00:24<00:00, 11.75it/s, bce_loss - 0.1092, iou_score - 0.5511] 

Epoch: 2
valid: 100%|██████████| 284/284 [00:24<00:00, 11.74it/s, bce_loss - 0.1373, iou_score - 0.5473]

Epoch: 3
valid: 100%|██████████| 284/284 [00:24<00:00, 11.73it/s, bce_loss - 0.1348, iou_score - 0.5029]

Epoch: 4
valid: 100%|██████████| 284/284 [00:24<00:00, 11.73it/s, bce_loss - 0.1159, iou_score - 0.5613] 

Epoch: 5
valid: 100%|██████████| 284/284 [00:24<00:00, 11.73it/s, bce_loss - 0.1115, iou_score - 0.521]  

Epoch: 6
valid: 100%|██████████| 284/284 [00:24<00:00, 11.75it/s, bce_loss - 0.1188, iou_score - 0.5364]

Epoch: 7
valid: 100%|██████████| 284/284 [00:24<00:00, 11.77it/s, bce_loss - 0.4422, iou_score - 0.43]  

Epoch: 8
valid: 100%|██████████| 284/284 [00:24<00:00, 11.72it/s, bce_loss - 0.8116, iou_score - 0.4239]

Epoch: 9
valid: 100%|██████████| 284/284 [00:24<00:0

In [29]:
torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '../forestnet/weights/forestnet_chekpoint_v1.2 - 0.69.pth')

In [35]:
torch.save(model, '../geonet/weights/forestnet_v1.2 - 0.69.pth')

## LightGBM classifier

In [39]:
import lightgbm as lgb

In [334]:
X_tr = utils.flatten_file("../../data/_mvp/liski_3857.tif")
Y_tr = raster.get_array_from_tiff("../../data/_mvp/liski_forest_mask.tif")[0].flatten()

In [335]:
print(X_tr.shape)
print(Y_tr.shape)

(123109938, 4)
(123109938,)


In [7]:
lgb_train = lgb.Dataset(X_tr, Y_tr)

In [8]:
params = {
    'task':'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'histogram_pool_size': 16,
    'min_data_in_leaf': 50
}

In [9]:
gbm = lgb.train(params, lgb_train, num_boost_round=70)

In [40]:
def refit(model, X, Y, decay_rate, chunk = 5000000):
    X = utils.flatten_file(X)
    Y = raster.get_array_from_tiff(Y)[0].flatten()
    steps = np.int(np.floor(X.shape[0] / chunk))
    first_step = 0
    for i in range(0, steps):
        model.refit(X[first_step:(first_step+chunk), :], Y[first_step:(first_step+chunk)], decay_rate)
        first_step += chunk
    return model

In [41]:
mod = lgb.Booster(model_file='../geonet/weights/best/forest_gbm_v0.1.2.txt')

In [5]:
mod = refit(mod, "../../data/_mvp/liski_3857.tif", "../../data/_mvp/liski_forest_mask.tif", decay_rate=0.9)

In [7]:
mod.save_model('../geonet/weights/forest_gbm_v0.1.3.txt')

In [15]:
del(img, mas)

In [6]:
img = raster.get_array_from_tiff("../../data/_mvp/vrn_19_coord.tif")
img = np.dstack(img)

In [7]:
mas = raster.get_array_from_tiff("../../data/_mvp/vrn_3857_forest_mask.tif")[0]

In [8]:
testData = dataset.RasterDataset(img, mas, classes=['agro'], tile_size=1024, step=256, preprocessing=get_inference_preprocessing(preprocessing_fn))

/home/DVL/glotovaa/projects/data_science/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


In [9]:
test_loader = DataLoader(testData, batch_size=1, shuffle=False, num_workers=1)

In [10]:
forest_model = torch.load('../geonet/weights/forestnet-c_bce_v1.3 - 0.6568863365000599.pth')

In [11]:
def cnn_predict(model, img, test_loader):
    ext_x = np.zeros(shape=(img.shape[0], img.shape[1]), dtype=np.float32)
    step = 256
    tile_size = 1024
    xc = round(img.shape[0] / step) + 1
    yc = round(img.shape[1] / step) + 1

    i = 0
    for batch in test_loader:
        m = i % xc
        j = i // xc
        #x_tensor = torch.from_numpy(batch[0]).unsqueeze(0)
        pr_mask = model.predict(batch[0].cuda())
        pr_mask = (pr_mask.cpu().numpy().round(decimals=2))

            
        if (step*m+tile_size) > img.shape[0]:
            if (step*j+tile_size) > img.shape[1]:
                ext_x[(img.shape[0]-tile_size):img.shape[0], (img.shape[1]-tile_size):img.shape[1]] = np.maximum(ext_x[(img.shape[0]-tile_size):img.shape[0], (img.shape[1]-tile_size):img.shape[1]], pr_mask)
            else:
                ext_x[(img.shape[0]-tile_size):img.shape[0], step*j:(step*j+tile_size)] = np.maximum(ext_x[(img.shape[0]-tile_size):img.shape[0], step*j:(step*j+tile_size)], pr_mask)
        elif (step*j+tile_size) > img.shape[1]:
            ext_x[step*m:(step*m+tile_size), (img.shape[1]-tile_size):img.shape[1]] = np.maximum(ext_x[step*m:(step*m+tile_size), (img.shape[1]-tile_size):img.shape[1]], pr_mask)
        else:
            ext_x[step*m:(step*m+tile_size), step*j:(step*j+tile_size)] = np.maximum(ext_x[step*m:(step*m+tile_size), step*j:(step*j+tile_size)], pr_mask)
    
        i += 1
    
    return ext_x

In [12]:
data = np.dstack(raster.get_array_from_tiff("../../data/_mvp/vrn_19_coord.tif"))
ext_x = cnn_predict(forest_model, data, test_loader)

In [13]:
raster.get_raster_from_array(ext_x, "../../data_science/mvp/vrn_19_forest_predicted_v.1.3-c_bce.tif", "../../data/_mvp/vrn_19_coord.tif")

In [13]:
def complex_predict(img):
    forest_model = torch.load('../geonet/weights/best/forestnet_v1.2 - 0.613902030818754.pth')
    image = raster.get_array_from_tiff(img)
    cnn_img = np.dstack(image)
    testData = dataset.RasterDataset(cnn_img.astype(float), cnn_img[0].astype(float), classes=['agro'], tile_size=1024, step=768, preprocessing=get_inference_preprocessing(preprocessing_fn))
    test_loader = DataLoader(testData, batch_size=1, shuffle=False, num_workers=1)
    gbm = lgb.Booster(model_file="../geonet/weights/forest_gbm_v0.1.3.txt")
    cnn_pred = cnn_predict(forest_model, cnn_img, test_loader)
    gbm_pred = gbm.predict(utils.flatten_file(img))
    gbm_pred = np.reshape(gbm_pred, image[0].shape)
    preds = np.maximum(cnn_pred, gbm_pred)
    return preds

In [14]:
d = complex_predict("../../data/_mvp/liski_3857.tif")

(11109, 11082)
(11109, 11082)
(11109, 11082)


In [35]:
gbm = lgb.Booster(model_file="../geonet/weights/forest_gbm_v0.1.3.txt")
gbm_pred = gbm.predict(utils.flatten_file("../../data/_mvp/vrn_train_3857.tif"))
image = raster.get_array_from_tiff("../../data/_mvp/vrn_train_3857.tif")
shape=image[0].shape
del(image)
gbm_pred = np.reshape(gbm_pred, shape)

In [33]:
ext_x = cnn_predict(forest_model, img, test_loader)

In [36]:
f = np.maximum(ext_x, gbm_pred)

In [37]:
raster.get_raster_from_array(f, "../mvp/vrn_forest_predicted_cpx_v2.tif", "../../data/_mvp/vrn_train_3857.tif")

In [38]:
del(ext_x, f, gbm_pred)